## Setup

In [1]:
import os

REPO_URL = "https://github.com/fabioantonacci79/BasicDSP.git"
REPO_NAME = "BasicDSP"

if not os.path.exists(REPO_NAME):
    !git clone {REPO_URL}

%cd BasicDSP/notebooks

# %%
# Microphone array notebook – wavefronts, GCC/PHAT, SRP-PHAT, aliasing, beamforming

import numpy as np
import matplotlib.pyplot as plt

from ipywidgets import interact, FloatSlider, IntSlider
from IPython.display import display

%matplotlib inline

c = 343.0  # speed of sound (m/s)
fs = 16000  # sampling rate (Hz)
rng = np.random.default_rng(0)

## Wavefront acquisition by a linear array

In [20]:
# %%
def simulate_wavefront(d=0.05, M=8, f0=1000.0, theta_deg=30.0):
    # θ measured from broadside (0° = frontal)
    theta = np.deg2rad(theta_deg)
    T = 5e-3  # 5 ms window
    N = int(T * fs)
    t = np.arange(N) / fs

    # time delays for ULA: τ_m(θ) = m d cos θ / c
    m = np.arange(M)
    tau = m * d * np.cos(theta) / c  # seconds

    # source waveform: a short windowed sinusoid
    s = np.cos(2 * np.pi * f0 * t) * np.hanning(N)

    x = np.zeros((M, N))
    S = np.fft.rfft(s)
    freqs = np.fft.rfftfreq(N, 1/fs)
    for i in range(M):
        # fractional delay via phase in frequency domain
        delay_phase = np.exp(-1j * 2 * np.pi * freqs * tau[i])
        Xi = S * delay_phase
        x[i] = np.fft.irfft(Xi)

    plt.figure(figsize=(8, 4))
    offset = 1.2
    for i in range(M):
        plt.plot(t * 1e3, x[i] + i * offset, lw=1)
    plt.xlabel("Time (ms)")
    plt.ylabel("Microphone index (stacked)")
    plt.title(f"Plane wave at θ = {theta_deg:.1f}° (broadside=0°), d = {d*100:.1f} cm, f₀ = {f0:.0f} Hz")
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()


interact(
    simulate_wavefront,
    d=FloatSlider(value=0.05, min=0.02, max=0.25, step=0.01, description='d (m)'),
    M=IntSlider(value=8, min=2, max=16, step=1, description='M'),
    f0=FloatSlider(value=1000.0, min=200.0, max=4000.0, step=100.0, description='f0 (Hz)'),
    theta_deg=FloatSlider(
        value=0.0,
        min=-90.0,
        max=90.0,
        step=5.0,
        description='θ (deg, broadside=0°)'
    )
);



interactive(children=(FloatSlider(value=0.05, description='d (m)', max=0.25, min=0.02, step=0.01), IntSlider(v…

## GCC vs. GCC-PHAT

In [15]:
# Corrected GCC / GCC-PHAT demo (replace previous helpers)

import numpy as np
import matplotlib.pyplot as plt

eps = 1e-12

def make_coloured_source(N, fs, f_cut=2000.0):
    """White noise lowpassed by a simple FIR to make it non-white."""
    w = rng.standard_normal(N + 256)
    t = np.arange(-64, 65)
    h = np.sinc(2 * f_cut / fs * t) * np.hamming(len(t))
    h /= np.sum(h)
    x = np.convolve(w, h, mode='same')
    return x[:N]

def make_two_mic_signals(delay_ms=2.0, noise_snr_db=20.0):
    """
    Create two signals x1 and x2 where x2 is a *delayed* version of the source s
    by delay_ms (positive -> x2 arrives later than x1).
    This version avoids np.roll wrap-around.
    """
    N = 4096
    s = make_coloured_source(N, fs)
    delay_samples = int(np.round(delay_ms * 1e-3 * fs))

    # create delayed version with zero padding (no wrap-around)
    if delay_samples >= 0:
        x1 = s.copy()
        x2 = np.concatenate((np.zeros(delay_samples), s[:N-delay_samples]))
    else:
        # negative delay: x2 advanced
        ds = -delay_samples
        x2 = s.copy()
        x1 = np.concatenate((np.zeros(ds), s[:N-ds]))

    # additive white noise (same power both channels)
    sig_power = np.mean(s**2)
    noise_power = sig_power / (10**(noise_snr_db/10))
    n1 = rng.normal(scale=np.sqrt(noise_power), size=N)
    n2 = rng.normal(scale=np.sqrt(noise_power), size=N)
    x1 += n1
    x2 += n2
    return x1, x2

def gcc(x, y, phat=False, nfft=None):
    """
    Compute generalized cross-correlation between x and y.
    Returns (lags_seconds, correlation_values) where lags are centered (negative .. positive).
    Uses FFT-then-IFFT with FFT shift for correct centering.
    """
    N = len(x)
    if nfft is None:
        nfft = 2 * N   # simple choice; can be increased for finer lag resolution

    X = np.fft.fft(x, n=nfft)
    Y = np.fft.fft(y, n=nfft)
    G = X * np.conj(Y)

    if phat:
        G = G / (np.abs(G) + eps)  # per-bin safe normalization (PHAT)
    # inverse FFT and center
    r = np.fft.ifft(G)
    r = np.real(np.fft.fftshift(r))  # center the zero-lag
    # lags in samples: -(nfft//2) ... (nfft//2 - 1)
    half = nfft // 2
    lags = np.arange(-half, -half + nfft) / fs
    return lags, r

def gcc_demo_fixed(delay_ms=2.0, noise_snr_db=20.0):
    x1, x2 = make_two_mic_signals(delay_ms, noise_snr_db)
    lags, r_gcc = gcc(x1, x2, phat=False)
    _, r_phat = gcc(x1, x2, phat=True)

    # find peak locations (in ms)
    idx_gcc = np.argmax(np.abs(r_gcc))
    idx_phat = np.argmax(np.abs(r_phat))
    lag_gcc_ms = lags[idx_gcc] * 1e3
    lag_phat_ms = lags[idx_phat] * 1e3

    plt.figure(figsize=(10,4))
    plt.plot(lags*1e3, r_gcc, label="GCC")
    plt.plot(lags*1e3, r_phat, label="GCC-PHAT")
    # true delay line: the way we constructed x2, x2 is delayed by +delay_ms (x2 arrives later)
    plt.axvline(delay_ms, color='k', ls='--', alpha=0.7, label='True delay (ms)')
    # estimated peaks
    plt.axvline(lag_gcc_ms, color='C0', ls=':', alpha=0.9, label=f'GCC peak {lag_gcc_ms:.2f} ms')
    plt.axvline(lag_phat_ms, color='C1', ls='-.', alpha=0.9, label=f'PHAT peak {lag_phat_ms:.2f} ms')

    plt.xlabel("Lag (ms)")
    plt.ylabel("Correlation")
    plt.title("GCC vs GCC-PHAT (corrected alignment & PHAT normalization)")
    plt.grid(True, alpha=0.3)
    plt.legend()
    plt.tight_layout()
    plt.show()

    print(f"GCC peak lag = {lag_gcc_ms:.3f} ms, PHAT peak lag = {lag_phat_ms:.3f} ms, true delay = {delay_ms:.3f} ms")


# Robust GCC / GCC-PHAT that auto-corrects sign/convention issues

import numpy as np
import matplotlib.pyplot as plt

eps = 1e-12

def gcc_auto(x, y, phat=False, nfft=None, expected_delay_ms=None):
    """
    Compute generalized cross-correlation between x and y, but check peak sign.
    If the peak sign disagrees with expected_delay_ms (non-zero), flip multiplication order.
    Returns (lags_seconds, correlation_values).
    """
    N = len(x)
    if nfft is None:
        nfft = 2 * N

    X = np.fft.fft(x, n=nfft)
    Y = np.fft.fft(y, n=nfft)

    # first try: X * conj(Y)
    G = X * np.conj(Y)
    if phat:
        G = G / (np.abs(G) + eps)

    r = np.fft.ifft(G)
    r = np.real(np.fft.fftshift(r))
    half = nfft // 2
    lags = np.arange(-half, -half + nfft) / fs

    # check peak sign if expected_delay_ms provided
    if expected_delay_ms is not None and expected_delay_ms != 0:
        idx_peak = np.argmax(np.abs(r))
        peak_lag_ms = lags[idx_peak] * 1e3
        # If sign disagree, try swapped ordering
        if np.sign(peak_lag_ms) != np.sign(expected_delay_ms):
            # recompute with conjugate order (conj(X)*Y)
            G2 = np.conj(X) * Y
            if phat:
                G2 = G2 / (np.abs(G2) + eps)
            r2 = np.fft.ifft(G2)
            r2 = np.real(np.fft.fftshift(r2))
            # choose the one whose peak sign matches expected_delay_ms
            idx2 = np.argmax(np.abs(r2))
            peak2_ms = lags[idx2] * 1e3
            if np.sign(peak2_ms) == np.sign(expected_delay_ms):
                r = r2
                # debug info (can be removed)
                print(f"[gcc_auto] swapped cross-spectrum ordering to match expected delay (peak {peak2_ms:.3f} ms).")
            else:
                # neither matched — keep original but warn
                print(f"[gcc_auto] warning: peak sign still disagrees (orig {peak_lag_ms:.3f} ms, swap {peak2_ms:.3f} ms).")
    return lags, r

def gcc_demo_autocorrect(delay_ms=2.0, noise_snr_db=20.0):
    x1, x2 = make_two_mic_signals(delay_ms, noise_snr_db)
    lags, r_gcc = gcc_auto(x1, x2, phat=False, expected_delay_ms=delay_ms)
    _, r_phat = gcc_auto(x1, x2, phat=True, expected_delay_ms=delay_ms)

    idx_gcc = np.argmax(np.abs(r_gcc))
    idx_phat = np.argmax(np.abs(r_phat))
    lag_gcc_ms = lags[idx_gcc] * 1e3
    lag_phat_ms = lags[idx_phat] * 1e3

    plt.figure(figsize=(10,4))
    plt.plot(lags*1e3, r_gcc, label="GCC")
    plt.plot(lags*1e3, r_phat, label="GCC-PHAT")
    plt.axvline(delay_ms, color='k', ls='--', alpha=0.8, label='True delay (ms)')
    plt.axvline(lag_gcc_ms, color='C0', ls=':', alpha=0.9, label=f'GCC peak {lag_gcc_ms:.2f} ms')
    plt.axvline(lag_phat_ms, color='C1', ls='-.', alpha=0.9, label=f'PHAT peak {lag_phat_ms:.2f} ms')

    plt.xlabel("Lag (ms)")
    plt.ylabel("Correlation")
    plt.title("GCC vs GCC-PHAT (auto-corrected sign)")
    plt.xlim(-100,100)
    plt.grid(True, alpha=0.3)
    plt.legend()
    plt.tight_layout()
    plt.show()

    print(f"GCC peak lag = {lag_gcc_ms:.3f} ms, PHAT peak lag = {lag_phat_ms:.3f} ms, true delay = {delay_ms:.3f} ms")


# Small interactive test
from ipywidgets import interact, FloatSlider
interact(gcc_demo_autocorrect,
         delay_ms=FloatSlider(value=2.0, min=-10.0, max=10.0, step=0.25, description='delay (ms)'),
         noise_snr_db=FloatSlider(value=20.0, min=0.0, max=40.0, step=2.0, description='SNR (dB)'));

interactive(children=(FloatSlider(value=2.0, description='delay (ms)', max=10.0, min=-10.0, step=0.25), FloatS…

## SRP-PHAT in 1D with ULA

In [2]:
# %% [markdown]
# ### SRP-PHAT (half-space scan to remove left/right ambiguity)
#
# This example restricts the SRP-PHAT scan to the half-space **θ ∈ [0°, 90°]**
# so that the ULA left/right ambiguity is removed for classroom demonstrations.
# (Angle convention: θ measured from broadside; 0° = broadside.)
#
# Sliders:
# - True θ (0..90°)
# - M = number of microphones
# - d = spacing (m)
# - T = analysis window (s)

# %%
# SRP-PHAT half-space demo (self-contained cell)

import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider, IntSlider
from IPython.display import display

# Ensure globals (if not defined previously)
try:
    fs
    c
    rng
except NameError:
    c = 343.0
    fs = 16000
    rng = np.random.default_rng(0)

def simulate_array_signals(theta_deg=20.0, M=8, d=0.05, T=0.05):
    """Simulate broadband source on ULA with additive sensor noise."""
    theta = np.deg2rad(theta_deg)
    N = int(T * fs)
    t = np.arange(N) / fs
    # broadband source (coloured by small window)
    s = rng.standard_normal(N-128)
    b = np.hanning(129); b /= np.sum(b)
    s = np.convolve(s, b, mode='full')
    m = np.arange(M)
    tau = m * d * np.cos(theta) / c
    X = np.zeros((M, N))
    S = np.fft.rfft(s)
    freqs = np.fft.rfftfreq(N, 1/fs)
    for i in range(M):
        phase = np.exp(-1j * 2 * np.pi * freqs * tau[i])
        Xi = S * phase
        xi = np.fft.irfft(Xi)
        xi += 0.001 * rng.standard_normal(N)  # sensor noise
        X[i] = xi
    return X

def compute_gcc_phat_pairs(X, nfft=None):
    """Compute pairwise GCC-PHAT and return dict {(i,j):(r,nfft)} with centered correlation."""
    M, N = X.shape
    if nfft is None:
        nfft = 2 * N - 1
    print(np.shape(X))
    Xf = np.fft.fft(X, n=nfft, axis=1)
    gcc_pairs = {}
    for i in range(M):
        for j in range(i+1, M):
            G = Xf[i] * np.conj(Xf[j])
            G = G / (np.abs(G) + 1e-12)  # PHAT normalization
            r = np.fft.ifft(G)
            r = np.real(np.fft.fftshift(r))  # center zero-lag
            gcc_pairs[(i, j)] = (r, nfft)
    return gcc_pairs

def srp_phat_scan_from_pairs(gcc_pairs, M, N, d, angle_grid_rad, fs_local=fs):
    """
    Compute SRP-PHAT from pairwise GCC-PHAT using fractional-lag interpolation.

    Uses tau_ij = (j-i)*d*cos(theta)/c, and evaluates r_ij at lag = tau_ij*fs
    via linear interpolation instead of rounding to an integer sample.
    """
    # assume same nfft for all pairs
    any_pair = next(iter(gcc_pairs))
    nfft = gcc_pairs[any_pair][1]
    half = nfft // 2

    P = np.zeros(len(angle_grid_rad), dtype=float)

    for k, ang in enumerate(angle_grid_rad):
        val = 0.0
        for (i, j), (r, _) in gcc_pairs.items():
            tau_ij = (j - i) * d * np.cos(ang) / c
            lag = tau_ij * fs_local  # fractional lag in samples

            x = lag   # fractional index into r (which is fftshifted)
            i0 = int(np.floor(x))
            a = x - i0  # fractional part
            #val += (r[i0])
            # linear interpolation r[x] ~ (1-a) r[i0] + a r[i0+1]
            if 0 <= i0 < len(r) - 1:
                val += (1.0 - a) * r[i0] + a * r[i0 + 1]
            # else: out of bounds -> ignore this pair for this angle

        P[k] = val

    return P


def srp_phat_halfspace_demo(true_theta=20.0, M=8, d=0.05, T=0.05, angle_resolution_deg=1.0):
    """
    Simulate signals and compute SRP-PHAT, scanning only the half-space [0, 90] degrees.
    """
    # simulate
    X = simulate_array_signals(theta_deg=true_theta, M=M, d=d, T=T)
    Mx, N = X.shape

    # compute pairwise GCC-PHAT
    gcc_pairs = compute_gcc_phat_pairs(X)

    # half-space angle grid (0..90 deg)
    angle_grid_deg = np.arange(0.0, 90.0 + angle_resolution_deg, angle_resolution_deg)
    angle_grid_rad = np.deg2rad(angle_grid_deg)

    # SRP-PHAT scan
    P = srp_phat_scan_from_pairs(gcc_pairs, Mx, N, d, angle_grid_rad, fs_local=fs)
    est_idx = np.argmax(P)
    est_doA = angle_grid_deg[est_idx]

    # plot
    plt.figure(figsize=(8,4))
    plt.plot(angle_grid_deg, P, '-o', lw=1, ms=4)
    plt.axvline(true_theta, color='g', ls='--', label='True DOA')
    plt.axvline(est_doA, color='r', ls='-.', label=f'Estimated DOA = {est_doA:.1f}°')
    plt.xlabel("Angle (deg) (restricted to [0°, 90°])")
    plt.ylabel("SRP-PHAT")
    plt.title("SRP-PHAT (half-space scan)")
    plt.grid(True, alpha=0.3)
    plt.legend()
    plt.tight_layout()
    plt.show()

# interactive widgets
interact(
    srp_phat_halfspace_demo,
    true_theta=FloatSlider(value=20.0, min=0.0, max=90.0, step=1.0, description='True θ (deg, 0..90)'),
    M=IntSlider(value=20, min=16, max=32, step=2, description='M'),
    d=FloatSlider(value=0.02, min=0.01, max=0.08, step=0.01, description='d (m)'),
    T=FloatSlider(value=0.15, min=0.1, max=0.20, step=0.01, description='T (s)'),
    angle_resolution_deg=FloatSlider(value=1.0, min=0.25, max=5.0, step=0.25, description='angle res (°)')
);


interactive(children=(FloatSlider(value=20.0, description='True θ (deg, 0..90)', max=90.0, step=1.0), IntSlide…

## Spatial aliasing of a sinusoidal wavefront

In [10]:
# %%
# 4) Linear array sampling a sinusoidal spatial wavefront, spacing slider

def spatial_sampling_demo(d=0.05, wavelength=0.2, M=8):
    k = 2 * np.pi / wavelength

    # continuous axis
    x_max = (M - 1) * d + wavelength
    x = np.linspace(0, x_max, 1200)

    # true spatial sinusoid
    p_true = np.cos(k * x)

    # aliased spatial sinusoid (guaranteed same samples at x_m = m d)
    k_alias = k  - 2 * np.pi / d
    p_alias = np.cos(k_alias * x)

    # samples at mic positions
    m = np.arange(M)
    xm = m * d
    pm = np.cos(k * xm)  # equals cos(k_alias * xm)

    plt.figure(figsize=(8, 4))
    plt.plot(x, p_true, lw=2, label=r"true: $\cos(kx)$")
    plt.plot(x, p_alias, lw=2, ls="--", label=r"aliased: $\cos((k+2\pi/d)x)$")
    plt.scatter(xm, pm, color="r", zorder=3, label="samples at mic positions")

    for xi in xm:
        plt.axvline(xi, color='k', lw=0.3, alpha=0.25)

    plt.xlabel("Position along array x (m)")
    plt.ylabel("Pressure (arb. units)")
    plt.title(f"Spatial aliasing: d = {d*100:.1f} cm, λ = {wavelength*100:.1f} cm")
    plt.grid(True, alpha=0.3)
    plt.legend()
    plt.tight_layout()
    plt.show()

    # optional: show alias-free frequency threshold
    f_alias = c / (2 * d)
    print(f"Nyquist-like condition: d ≤ λ/2  ⇔  f ≤ c/(2d) ≈ {f_alias:.1f} Hz")


interact(
    spatial_sampling_demo,
    d=FloatSlider(value=0.05, min=0.02, max=0.5, step=0.01, description='d (m)'),
    wavelength=FloatSlider(value=0.2, min=0.05, max=1.0, step=0.01, description='λ (m)'),
    M=IntSlider(value=8, min=3, max=16, step=1, description='M')
);


interactive(children=(FloatSlider(value=0.05, description='d (m)', max=0.5, min=0.02, step=0.01), FloatSlider(…

## DAS and MVDR beamformers (desired source + diffuse noise)

In [17]:
# %% [markdown]
# ## 5. Delay-and-sum vs MVDR beamformers and pseudospectrum (single source + diffuse noise)
#
# Narrowband setting at frequency f0.
# - Desired source at θ_sig
# - Additive diffuse white noise (identity covariance)
#
# We compare:
# - Delay-and-sum (CBF) beampattern
# - MVDR beampattern
# - MVDR pseudospectrum  P(θ) = 1 / (a^H R^{-1} a)

# %%
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider, IntSlider

# Ensure globals exist
try:
    fs
    c
except NameError:
    c = 343.0
    fs = 16000

def steering_vector(theta_deg, M=8, d=0.05, f0=2000.0):
    """ULA steering vector, theta measured from broadside (0° = broadside)."""
    theta = np.deg2rad(theta_deg)
    k = 2 * np.pi * f0 / c
    m = np.arange(M)
    return np.exp(-1j * k * m * d * np.cos(theta))

def make_covariance_single_source(theta_sig_deg=0.0,
                                  M=8, d=0.05, f0=2000.0,
                                  snr_sig_db=10.0,
                                  noise_power=1.0):
    """
    Spatial covariance for a single narrowband source + diffuse white noise:
      R = Ps a a^H + sigma^2 I
    where Ps is set from snr_sig_db relative to noise_power.
    """
    a_s = steering_vector(theta_sig_deg, M, d, f0)[:, None]
    Ps = noise_power * 10**(snr_sig_db / 10)
    R = Ps * (a_s @ a_s.conj().T) + noise_power * np.eye(M)
    return R

def beampatterns_and_pseudospectrum_single_source(theta_sig_deg=0.0,
                                                  M=8, d=0.05, f0=2000.0,
                                                  snr_sig_db=10.0):
    R = make_covariance_single_source(theta_sig_deg, M, d, f0, snr_sig_db, noise_power=1.0)
    Rinv = np.linalg.inv(R)

    # Conventional (delay-and-sum) weights steered to desired DOA
    a0 = steering_vector(theta_sig_deg, M, d, f0)
    w_cbf = a0 / M

    # MVDR weights steered to desired DOA
    w_mvdr = (Rinv @ a0) / (a0.conj().T @ Rinv @ a0)

    angles = np.linspace(0, 180, 721)

    B_cbf = np.zeros_like(angles, dtype=float)
    B_mvdr = np.zeros_like(angles, dtype=float)
    P_mvdr = np.zeros_like(angles, dtype=float)
    P_cbf = np.zeros_like(angles, dtype=float)

    for i, ang in enumerate(angles):
        a = steering_vector(ang, M, d, f0)
        B_cbf[i] = np.abs(w_cbf.conj().T @ a)
        B_mvdr[i] = np.abs(w_mvdr.conj().T @ a)
        P_mvdr[i] = 1.0 / np.real(a.conj().T @ Rinv @ a)
        P_cbf[i] = np.abs((a.conj().T @ R @ a))

    # Normalize to 0 dB peak
    B_cbf_db = 20 * np.log10(B_cbf / np.max(B_cbf))
    B_mvdr_db = 20 * np.log10(B_mvdr / np.max(B_mvdr))
    P_mvdr_db = 10 * np.log10(P_mvdr / np.max(P_mvdr))
    P_cbf_db = 10 * np.log10(P_cbf / np.max(P_cbf))
    plt.figure(figsize=(8, 5))
    plt.plot(angles, B_cbf_db, label='Delay-and-sum beampattern')
    plt.plot(angles, B_mvdr_db, label='MVDR beampattern')
    plt.plot(angles, P_mvdr_db, ls='--', label='MVDR pseudospectrum')
    plt.plot(angles, P_cbf_db,ls='--', label='DAS pseudospectrum')
    plt.axvline(theta_sig_deg, color='g', ls=':', label='Desired DOA')

    plt.ylim(-60, 5)
    plt.xlabel("Angle (deg)")
    plt.ylabel("Normalized response (dB)")
    plt.title(f"CBF vs MVDR (single source, f0={f0:.0f} Hz, M={M}, d={d*100:.1f} cm)")
    plt.grid(True, alpha=0.3)
    plt.legend(loc='lower left', fontsize=8)
    plt.tight_layout()
    plt.show()


interact(
    beampatterns_and_pseudospectrum_single_source,
    theta_sig_deg=FloatSlider(value=20.0, min=0.0, max=180.0, step=5.0, description='Signal θ (deg)'),
    M=IntSlider(value=8, min=4, max=16, step=2, description='M'),
    d=FloatSlider(value=0.05, min=0.02, max=0.25, step=0.01, description='d (m)'),
    f0=FloatSlider(value=2000.0, min=500.0, max=4000.0, step=100.0, description='f0 (Hz)'),
    snr_sig_db=FloatSlider(value=10.0, min=-5.0, max=30.0, step=1.0, description='SNR (dB)')
);


interactive(children=(FloatSlider(value=20.0, description='Signal θ (deg)', max=180.0, step=5.0), IntSlider(va…

 ## Delay-and-sum vs MVDR beamformers + pseudospectrum (interferer + source + diffuse noise)

In [19]:
# %%
# 6) Delay-and-sum vs MVDR beamformers, with diffuse white noise

def steering_vector(theta_deg, M=8, d=0.05, f0=2000.0):
    theta = np.deg2rad(theta_deg)
    k = 2 * np.pi * f0 / c
    m = np.arange(M)
    return np.exp(-1j * k * m * d * np.cos(theta))

def make_covariance(theta_sig_deg=0.0, theta_int_deg=40.0,
                    M=8, d=0.05, f0=2000.0,
                    snr_sig_db=10.0, snr_int_db=20.0,
                    noise_power=1.0):
    a_s = steering_vector(theta_sig_deg, M, d, f0)[:, None]
    a_i = steering_vector(theta_int_deg, M, d, f0)[:, None]

    # set signal/interferer powers relative to diffuse noise
    Ps = noise_power * 10**(snr_sig_db / 10)
    Pi = noise_power * 10**(snr_int_db / 10)

    R = Ps * (a_s @ a_s.conj().T) + Pi * (a_i @ a_i.conj().T) + noise_power * np.eye(M)
    return R

def beampatterns_and_pseudospectrum(theta_sig_deg=0.0, theta_int_deg=40.0,
                                    M=8, d=0.05, f0=2000.0,
                                    snr_sig_db=10.0, snr_int_db=20.0):
    R = make_covariance(theta_sig_deg, theta_int_deg, M, d, f0,
                        snr_sig_db, snr_int_db, noise_power=1.0)
    Rinv = np.linalg.inv(R)

    # Conventional beamformer weights (delay-and-sum toward theta_sig_deg)
    a0 = steering_vector(theta_sig_deg, M, d, f0)
    w_cbf = a0 / M

    # MVDR weights
    w_mvdr = (Rinv @ a0) / (a0.conj().T @ Rinv @ a0)

    angles = np.linspace(0, 180, 721)
    B_cbf = np.zeros_like(angles, dtype=float)
    B_mvdr = np.zeros_like(angles, dtype=float)
    P_mvdr = np.zeros_like(angles, dtype=float)
    P_cbf = np.zeros_like(angles, dtype=float)
    for i, ang in enumerate(angles):
        a = steering_vector(ang, M, d, f0)
        B_cbf[i] = np.abs(w_cbf.conj().T @ a)
        B_mvdr[i] = np.abs(w_mvdr.conj().T @ a)
        P_mvdr[i] = 1.0 / np.real(a.conj().T @ Rinv @ a)
        P_cbf[i] = np.abs((a.conj().T @ R @ a))
    # normalize
    B_cbf_db = 20 * np.log10(B_cbf / np.max(B_cbf))
    B_mvdr_db = 20 * np.log10(B_mvdr / np.max(B_mvdr))
    P_mvdr_db = 10 * np.log10(P_mvdr / np.max(P_mvdr))
    P_cbf_db = 20 * np.log10(P_cbf / np.max(P_cbf))
    plt.figure(figsize=(8,5))
    plt.plot(angles, B_cbf_db, label='Delay-and-sum beampattern')
    plt.plot(angles, B_mvdr_db, label='MVDR beampattern')
    plt.plot(angles, P_mvdr_db, ls='--', label='MVDR pseudospectrum (1/aᵀR⁻¹a)')
    plt.plot(angles, P_cbf_db, ls='--', label='DAS pseudospectrum')
    plt.axvline(theta_sig_deg, color='g', ls=':', label='Desired DOA')
    plt.axvline(theta_int_deg, color='r', ls='-.', label='Interferer DOA')

    plt.ylim(-60, 5)
    plt.xlabel("Angle (deg)")
    plt.ylabel("Normalized response (dB)")
    plt.title(f"CBF vs MVDR (f0={f0:.0f} Hz, M={M}, d={d*100:.1f} cm)")
    plt.grid(True, alpha=0.3)
    plt.legend(loc='lower left', fontsize=8)
    plt.tight_layout()
    plt.show()


interact(
    beampatterns_and_pseudospectrum,
    theta_sig_deg=FloatSlider(value=20.0, min=0.0, max=180.0, step=5.0, description='Signal θ (deg)'),
    theta_int_deg=FloatSlider(value=40.0, min=0.0, max=180.0, step=5.0, description='Interf θ (deg)'),
    M=IntSlider(value=8, min=4, max=16, step=2, description='M'),
    d=FloatSlider(value=0.05, min=0.02, max=0.25, step=0.01, description='d (m)'),
    f0=FloatSlider(value=2000.0, min=500.0, max=4000.0, step=100.0, description='f0 (Hz)'),
    snr_sig_db=FloatSlider(value=10.0, min=-5.0, max=30.0, step=1.0, description='SNR sig (dB)'),
    snr_int_db=FloatSlider(value=20.0, min=0.0, max=40.0, step=1.0, description='SNR int (dB)')
);


interactive(children=(FloatSlider(value=20.0, description='Signal θ (deg)', max=180.0, step=5.0), FloatSlider(…